In [2]:
import pandas as pd

# Read the four CSV files into Pandas DataFrames
df_f1_part1 = pd.read_csv('f1_part1.csv')
df_f1_part2 = pd.read_csv('f1_part2.csv')
df_f2_part1 = pd.read_csv('f2_part1.csv')
df_f2_part2 = pd.read_csv('f2_part2.csv')

# Concatenate the DataFrames vertically
combined_df = pd.concat([df_f1_part1, df_f1_part2, df_f2_part1, df_f2_part2], ignore_index=True)

# Display the columns of the combined DataFrame
print("Columns of the combined DataFrame:")
print(combined_df.columns)


Columns of the combined DataFrame:
Index(['timestamp_call_key', 'retailer_code', 'serial', 'reason', 'mos',
       'resolved', 'no_of_accounts_with_syf_13_march',
       'account_balance_13_march', 'delinquency_history_13_march',
       'account_open_date_13_march', 'account_status_13_march',
       'card_activation_status_13_march', 'eservice_ind_13_march',
       'ebill_enrolled_status_13_march', 'auto_pay_enrolled_status_13_march',
       'no_of_accounts_with_syf_18_march', 'account_balance_18_march',
       'delinquency_history_18_march', 'account_open_date_18_march',
       'account_status_18_march', 'card_activation_status_18_march',
       'eservice_ind_18_march', 'ebill_enrolled_status_18_march',
       'auto_pay_enrolled_status_18_march'],
      dtype='object')


In [3]:
combined_df.shape

(1798798, 24)

In [4]:
nan_counts = combined_df.isna().sum()

# Print out the number of NaN values for each column
print("Number of NaN values in each column:")
print(nan_counts)

Number of NaN values in each column:
timestamp_call_key                         0
retailer_code                              0
serial                                     0
reason                                     0
mos                                        0
resolved                                   0
no_of_accounts_with_syf_13_march        3400
account_balance_13_march              113782
delinquency_history_13_march               0
account_open_date_13_march                 0
account_status_13_march              1732768
card_activation_status_13_march            0
eservice_ind_13_march                      0
ebill_enrolled_status_13_march          4936
auto_pay_enrolled_status_13_march          0
no_of_accounts_with_syf_18_march        3402
account_balance_18_march              101829
delinquency_history_18_march               0
account_open_date_18_march                 0
account_status_18_march              1703450
card_activation_status_18_march            0
eservice_ind_18_ma

In [6]:
# combined_df["card_activation_status_13_march"].unique()
combined_df["mos"].unique()
len(combined_df) - len(combined_df["serial"].unique())

# resolved_count = (combined_df["resolved"] == 'resolved').sum()

290795

# EDA ON MOS

In [7]:
reason_codes = combined_df['mos'].str.split()

# Create a set of unique reason codes
unique_reasons = set(reason for sublist in reason_codes for reason in sublist)


In [28]:
# for reason in unique_reasons:
#     combined_df[reason] = combined_df['mos'].str.contains(reason).astype(int)

# reason EDA

In [8]:
from scipy.stats import chi2_contingency

# Example categorical column (categorical_variable) and binary variable (binary_variable)
categorical_variable = combined_df['reason']
binary_variable = combined_df['resolved']

# Chi-square test
chi2, p_value_chi2, _, _ = chi2_contingency(pd.crosstab(categorical_variable, binary_variable))
print("Chi-square test p-value:", p_value_chi2)


Chi-square test p-value: 0.0


In [9]:

# Create a cross-tabulation of the two variables
cross_tab = pd.crosstab(categorical_variable, binary_variable)

# Display the contingency table
print("Contingency table (Cross-tabulation):")
print(cross_tab)

Contingency table (Cross-tabulation):
resolved   floor  resolved
reason                    
AA         15089     39772
AP          4541      2188
AT          8687     82717
AU             9         7
BA        158394    407957
...          ...       ...
mo             1         0
mp          5426      2188
mt             3         2
ss             1         0
wa         13248      1048

[73 rows x 2 columns]


In [10]:
data = combined_df[['reason', 'resolved']]

In [24]:
resolved_counts = data[data['resolved'] == 'resolved'].groupby('reason').size().reset_index(name='resolved_count')
unresolved_counts = data[data['resolved'] == 'floor'].groupby('reason').size().reset_index(name='unresolved_count')

# print("Counts of resolved cases for each reason:")
print(resolved_counts)

# print("\nCounts of unresolved cases for each reason:")
# print(unresolved_counts)

   reason  resolved_count
0      AA           39772
1      AP            2188
2      AT           82717
3      AU               7
4      BA          407957
..    ...             ...
63     mm               1
64     mn            1226
65     mp            2188
66     mt               2
67     wa            1048

[68 rows x 2 columns]


In [33]:
combined_counts = pd.concat([resolved_counts.set_index('reason'), unresolved_counts.set_index('reason')], axis=1)
combined_counts.fillna(0, inplace=True)



In [34]:
tr_rows = combined_counts[combined_counts.index == 'ss']
print(tr_rows)



        resolved_count  unresolved_count
reason                                  
ss                 0.0               1.0


In [35]:
combined_counts['unresolved_to_total_ratio'] = combined_counts['unresolved_count'] / (combined_counts['resolved_count'] + combined_counts['unresolved_count'])

print("Ratio of unresolved to total for each reason category:")
print(combined_counts)

Ratio of unresolved to total for each reason category:
        resolved_count  unresolved_count  unresolved_to_total_ratio
reason                                                             
AA             39772.0           15089.0                   0.275041
AP              2188.0            4541.0                   0.674840
AT             82717.0            8687.0                   0.095040
AU                 7.0               9.0                   0.562500
BA            407957.0          158394.0                   0.279675
...                ...               ...                        ...
HI                 0.0               1.0                   1.000000
NU                 0.0               2.0                   1.000000
RS                 0.0              72.0                   1.000000
mo                 0.0               1.0                   1.000000
ss                 0.0               1.0                   1.000000

[73 rows x 3 columns]


In [39]:
# Sort the rows by unresolved_to_total_ratio in descending order
combined_counts_sorted = combined_counts.sort_values(by='unresolved_to_total_ratio', ascending=False)

print("Sorted DataFrame by unresolved_to_total_ratio (descending):")
combined_counts_sorted.head(50)


Sorted DataFrame by unresolved_to_total_ratio (descending):


,resolved_count,unresolved_count,unresolved_to_total_ratio
reason,,,
ss,0.0,1.0,1.000000
NU,0.0,2.0,1.000000
mo,0.0,1.0,1.000000
RS,0.0,72.0,1.000000
HI,0.0,1.0,1.000000
DP,102.0,6828.0,0.985281
LW,1.0,57.0,0.982759
DE,7.0,275.0,0.975177
CS,2.0,69.0,0.971831


In [65]:
# Filter rows where unresolved_count is greater than 1000
unresolved_count_gt_4000 = combined_counts_sorted[(combined_counts_sorted['unresolved_count'] > 1000)]

print("Rows where unresolved_count is greater than 1000:")
unresolved_count_gt_4000.head(20)

Rows where unresolved_count is greater than 1000:


,resolved_count,unresolved_count,unresolved_to_total_ratio
reason,,,
DP,102.0,6828.0,0.985281
CT,714.0,23082.0,0.969995
FD,533.0,12474.0,0.959022
FM,300.0,4455.0,0.936909
RV,263.0,3493.0,0.929979
wa,1048.0,13248.0,0.926693
me,217.0,1933.0,0.899070
iR,407.0,1755.0,0.811748
LC,2494.0,7582.0,0.752481


# digitally active